Training and Testing

In [ ]:
# Import Libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import time
from torch.quantization import quantize_dynamic

# Specify transforms using torchvision.transforms as transforms library
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load in each dataset and apply transformations using the torchvision.datasets as datasets library
train_set = datasets.ImageFolder("RDDC_Train", transform=transformations)
val_set = datasets.ImageFolder("RDDC_Test", transform=transformations)

print('The class labels are:', train_set.classes, '\n')

# Put into a Dataloader using torch library
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

# Get pretrained model using torchvision.models as models library
model = models.densenet161(pretrained=True)

# Turn off training for their parameters
for param in model.parameters():
    param.requires_grad = False

# Create new classifier for model using torch.nn as nn library
classifier_input = model.classifier.in_features
# classifier_input = 2208
print(classifier_input)

num_labels = 8
classifier = nn.Sequential(nn.Linear(classifier_input, 64),
                           nn.ReLU(),
                           nn.Linear(64, 32),
                           nn.ReLU(),
                           nn.Linear(32, num_labels),
                           nn.LogSoftmax(dim=1))

# Replace default classifier with new classifier
model.classifier = classifier

# Find the device available to use using torch library
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

# Move model to the device specified above
model.to(device)

# Set the error function using torch.nn as nn library
criterion = nn.NLLLoss()
# Set the optimizer function using torch.optim as optim library
optimizer = optim.Adam(model.classifier.parameters())

# Training the Model
epochs = 6
start_train = time.time()
epoch_array = []
accu_array = []
D00_TPR_array = []
D00_FPR_array = []
D01_TPR_array = []
D01_FPR_array = []
D10_TPR_array = []
D10_FPR_array = []
D11_TPR_array = []
D11_FPR_array = []
D20_TPR_array = []
D20_FPR_array = []
D40_TPR_array = []
D40_FPR_array = []
D43_TPR_array = []
D43_FPR_array = []
D44_TPR_array = []
D44_FPR_array = []
D00_TPR_array.append(0)
D00_FPR_array.append(0)
D01_TPR_array.append(0)
D01_FPR_array.append(0)
D10_TPR_array.append(0)
D10_FPR_array.append(0)
D11_TPR_array.append(0)
D11_FPR_array.append(0)
D20_TPR_array.append(0)
D20_FPR_array.append(0)
D40_TPR_array.append(0)
D40_FPR_array.append(0)
D43_TPR_array.append(0)
D43_FPR_array.append(0)
D44_TPR_array.append(0)
D44_FPR_array.append(0)

for epoch in range(epochs):
    display_epoch = epoch + 1
    print('Epoch:', display_epoch)
    epoch_array.append(display_epoch)
    train_loss = 0
    val_loss = 0
    accuracy = 0

    # Training the model
    model.train()
    counter = 0
    for inputs, labels in train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Clear optimizers
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(inputs)
        # Loss
        loss = criterion(output, labels)
        # Calculate gradients (backpropogation)
        loss.backward()
        # Adjust parameters based on gradients
        optimizer.step()
        # Add the loss to the training set's rnning loss
        train_loss += loss.item() * inputs.size(0)

        # Print the progress of our training
        counter += 1
        print("Batch:", counter, "out of", len(train_loader))

    end_train = time.time()
    print('Finished Epoch', epoch + 1, 'Training in %0.2f minutes' % ((end_train - start_train) / 60))

    # Evaluating the model
    start_valid = time.time()
    model.eval()
    counter = 0

    total_classes = 8
    output = torch.randn(batch_size, total_classes)  # refer to output after softmax
    target = torch.randint(0, total_classes, (batch_size,))  # labels
    confusion_matrix = torch.zeros(total_classes, total_classes)

    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            output = model.forward(inputs)
            # Calculate Loss
            valloss = criterion(output, labels)
            # Add loss to the validation set's running loss
            val_loss += valloss.item() * inputs.size(0)

            # Since our model outputs a LogSoftmax, find the real
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            # Print the progress of our evaluation
            counter += 1
            print("Batch:", counter, "out of", len(val_loader))

            _, preds = torch.max(output, 1)

            for p, t in zip(preds.view(-1), labels.view(-1)):
                confusion_matrix[p.long(), t.long()] += 1

        print(confusion_matrix)

        TP = confusion_matrix.diag()

        for c in range(total_classes):
            idx = torch.ones(total_classes).byte()
            idx[c] = 0
            TN = confusion_matrix[idx.nonzero()[:, None], idx.nonzero()].sum()
            FP = confusion_matrix[c, idx].sum()
            FN = confusion_matrix[idx, c].sum()

            sensitivity = (TP[c] / (TP[c] + FN))
            specificity = (TN / (TN + FP))
            FPR = 1 - specificity
            # re_call = (TP[c] / (TP[c] + FP))
            pre_cision = (TP[c] / (TP[c] + FN))
            f1_score = 2 * ((pre_cision * sensitivity) / (pre_cision + sensitivity))
            checking_c = c + 1
            if (checking_c == 1):
                D00_TPR_array.append(sensitivity)
                D00_FPR_array.append(FPR)
            elif (checking_c == 2):
                D01_TPR_array.append(sensitivity)
                D01_FPR_array.append(FPR)
            elif (checking_c == 3):
                D10_TPR_array.append(sensitivity)
                D10_FPR_array.append(FPR)
            elif (checking_c == 4):
                D11_TPR_array.append(sensitivity)
                D11_FPR_array.append(FPR)
            elif (checking_c == 5):
                D20_TPR_array.append(sensitivity)
                D20_FPR_array.append(FPR)
            elif (checking_c == 6):
                D40_TPR_array.append(sensitivity)
                D40_FPR_array.append(FPR)
            elif (checking_c == 7):
                D43_TPR_array.append(sensitivity)
                D43_FPR_array.append(FPR)
            else:
                D44_TPR_array.append(sensitivity)
                D44_FPR_array.append(FPR)

            print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(c + 1, TP[c], TN, FP, FN))
            print('Sensitivity or Recall = {}'.format(sensitivity))
            print('Specificity = {}'.format(specificity))
            # print('Recall = {}'.format(re_call))
            print('Precision = {}'.format(pre_cision))
            print('F1 Score = {}'.format(f1_score))

    end_valid = time.time()
    print('Finished Epoch', epoch + 1, 'Validating in %0.2f minutes' % ((end_valid - start_valid) / 60))

    # Get the average loss for the entire epoch
    train_loss = train_loss / len(train_loader.dataset)
    valid_loss = val_loss / len(val_loader.dataset)

    record_accuracy = (accuracy / len(val_loader)) * 100
    print("Record Accuracy", record_accuracy, " in epoch", display_epoch)
    accu_array.append(record_accuracy)

    # Print out the information
    print('Accuracy: %0.3f %%' % (accuracy / len(val_loader) * 100))
    print('Training Loss: {:.6f} ' '\tValidation Loss: {:.6f}'.format(train_loss, valid_loss), '\n')

    print('Total Time is %0.2f minutes' % ((end_valid - start_train) / 60))
import json
from pathlib import Path

# Create model directory
model_dir = Path("saved_model")
model_dir.mkdir(exist_ok=True)

# 1. Save full model (architecture + weights)
torch.save(model, model_dir/'road_damage_model.pth')

# 2. Save state dict (recommended for production)
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_to_idx': train_set.class_to_idx,
}, model_dir/'model_state.pth')

# 3. Save class labels
with open(model_dir/'class_labels.json', 'w') as f:
    json.dump(train_set.classes, f)

# 4. Save transformation parameters
transform_config = {
    'resize': 255,
    'crop': 224,
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}
with open(model_dir/'transform_config.json', 'w') as f:
    json.dump(transform_config, f)

# 5. Save model architecture details
model_architecture = {
    'base_model': 'densenet161',
    'classifier': {
        'input_features': classifier_input,
        'layers': [64, 32, num_labels],
        'activation': 'ReLU',
        'output_activation': 'LogSoftmax'
    }
}
with open(model_dir/'model_architecture.json', 'w') as f:
    json.dump(model_architecture, f)


print(f"\nModel and assets saved in {model_dir} directory")
D00_TPR_array.append(1)
D00_FPR_array.append(1)
D01_TPR_array.append(1)
D01_FPR_array.append(1)
D10_TPR_array.append(1)
D10_FPR_array.append(1)
D11_TPR_array.append(1)
D11_FPR_array.append(1)
D20_TPR_array.append(1)
D20_FPR_array.append(1)
D40_TPR_array.append(1)
D40_FPR_array.append(1)
D43_TPR_array.append(1)
D43_FPR_array.append(1)
D44_TPR_array.append(1)
D44_FPR_array.append(1)

D00_TPR_array.sort()
D00_FPR_array.sort()
D01_TPR_array.sort()
D01_FPR_array.sort()
D10_TPR_array.sort()
D10_FPR_array.sort()
D11_TPR_array.sort()
D11_FPR_array.sort()
D20_TPR_array.sort()
D20_FPR_array.sort()
D40_TPR_array.sort()
D40_FPR_array.sort()
D43_TPR_array.sort()
D43_FPR_array.sort()
D44_TPR_array.sort()
D44_FPR_array.sort()

# Title
plt.title('ROC Plot for DenseNet161')
# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot(D00_FPR_array, D00_TPR_array, color='blue', linewidth=3, label='D00')
plt.plot(D01_FPR_array, D01_TPR_array, color='green', linewidth=3, label='D01')
plt.plot(D10_FPR_array, D10_TPR_array, color='orange', linewidth=3, label='D10')
plt.plot(D11_FPR_array, D11_TPR_array, color='black', linewidth=3, label='D11')
plt.plot(D20_FPR_array, D20_TPR_array, color='cyan', linewidth=3, label='D20')
plt.plot(D40_FPR_array, D40_TPR_array, color='lavender', linewidth=3, label='D40')
plt.plot(D43_FPR_array, D43_TPR_array, color='lime', linewidth=3, label='D43')
plt.plot(D44_FPR_array, D44_TPR_array, color='coral', linewidth=3, label='D44')
plt.legend()
plt.show()

# Title
plt.title('Epoch Vs Accuracy for DenseNet161')
# Axis labels
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(epoch_array, accu_array)
plt.show()

The class labels are: ['D00', 'D01', 'D10', 'D11', 'D20', 'D40', 'D43', 'D44'] 



C:\Users\anupa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\anupa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2208
Epoch: 1
Batch: 1 out of 296
Batch: 2 out of 296
Batch: 3 out of 296
Batch: 4 out of 296
Batch: 5 out of 296
Batch: 6 out of 296
Batch: 7 out of 296
Batch: 8 out of 296
Batch: 9 out of 296
Batch: 10 out of 296
Batch: 11 out of 296
Batch: 12 out of 296
Batch: 13 out of 296
Batch: 14 out of 296
Batch: 15 out of 296
Batch: 16 out of 296
Batch: 17 out of 296
Batch: 18 out of 296
Batch: 19 out of 296
Batch: 20 out of 296
Batch: 21 out of 296
Batch: 22 out of 296
Batch: 23 out of 296
Batch: 24 out of 296
Batch: 25 out of 296
Batch: 26 out of 296
Batch: 27 out of 296
Batch: 28 out of 296
Batch: 29 out of 296
Batch: 30 out of 296
Batch: 31 out of 296
Batch: 32 out of 296
Batch: 33 out of 296
Batch: 34 out of 296
Batch: 35 out of 296
Batch: 36 out of 296
Batch: 37 out of 296
Batch: 38 out of 296
Batch: 39 out of 296
Batch: 40 out of 296
Batch: 41 out of 296
Batch: 42 out of 296
Batch: 43 out of 296
Batch: 44 out of 296
Batch: 45 out of 296
Batch: 46 out of 296
Batch: 47 out of 296
Batch: 4

C:\Users\anupa\AppData\Local\Temp\ipykernel_1920\2182347492.py:192: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen/native/IndexingUtils.h:30.)
  FP = confusion_matrix[c, idx].sum()
C:\Users\anupa\AppData\Local\Temp\ipykernel_1920\2182347492.py:193: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen/native/IndexingUtils.h:30.)
  FN = confusion_matrix[idx, c].sum()


Batch: 1 out of 296
Batch: 2 out of 296
Batch: 3 out of 296
Batch: 4 out of 296
Batch: 5 out of 296
Batch: 6 out of 296
Batch: 7 out of 296
Batch: 8 out of 296
Batch: 9 out of 296
Batch: 10 out of 296
Batch: 11 out of 296
Batch: 12 out of 296
Batch: 13 out of 296
Batch: 14 out of 296
Batch: 15 out of 296
Batch: 16 out of 296
Batch: 17 out of 296
Batch: 18 out of 296
Batch: 19 out of 296
Batch: 20 out of 296
Batch: 21 out of 296
Batch: 22 out of 296
Batch: 23 out of 296
Batch: 24 out of 296
Batch: 25 out of 296
Batch: 26 out of 296
Batch: 27 out of 296
Batch: 28 out of 296
Batch: 29 out of 296
Batch: 30 out of 296
Batch: 31 out of 296
Batch: 32 out of 296
Batch: 33 out of 296
Batch: 34 out of 296
Batch: 35 out of 296
Batch: 36 out of 296
Batch: 37 out of 296
Batch: 38 out of 296
Batch: 39 out of 296
Batch: 40 out of 296
Batch: 41 out of 296
Batch: 42 out of 296
Batch: 43 out of 296
Batch: 44 out of 296
Batch: 45 out of 296
Batch: 46 out of 296
Batch: 47 out of 296
Batch: 48 out of 296
B

In [3]:
import json
from pathlib import Path

# Create model directory
model_dir = Path("saved_model")
model_dir.mkdir(exist_ok=True)

# 1. Save full model (architecture + weights)
torch.save(model, model_dir/'road_damage_model.pth')

# 2. Save state dict (recommended for production)
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_to_idx': train_set.class_to_idx,
}, model_dir/'model_state.pth')

# 3. Save class labels
with open(model_dir/'class_labels.json', 'w') as f:
    json.dump(train_set.classes, f)

# 4. Save transformation parameters
transform_config = {
    'resize': 255,
    'crop': 224,
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}
with open(model_dir/'transform_config.json', 'w') as f:
    json.dump(transform_config, f)

# 5. Save model architecture details
model_architecture = {
    'base_model': 'densenet161',
    'classifier': {
        'input_features': classifier_input,
        'layers': [64, 32, num_labels],
        'activation': 'ReLU',
        'output_activation': 'LogSoftmax'
    }
}
with open(model_dir/'model_architecture.json', 'w') as f:
    json.dump(model_architecture, f)

print(f"\nModel and assets saved in {model_dir} directory")


Model and assets saved in saved_model directory


In [ ]:
# Quantize the model
quantized_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

# Save quantized model (expected size < 100MB)
torch.save(quantized_model.state_dict(), "saved_model/quantized_model.pth")